In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE126308"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE126308"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE126308.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE126308.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE126308.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Biomarkers in older ovarian cancer patients"
!Series_summary	"Identification and validation of potential prognostic biomarkers in older ovarian cancer patients with high-grade serous adenocarcinoma (HGSC)"
!Series_overall_design	"Biomarker study with the purpose of identifying and validating biomarkers in patients with early disease progression of HGSC vs. patients with late."
Sample Characteristics Dictionary:
{0: ['gender: female'], 1: ['tissue: ovarian tumor biopsy'], 2: ['diagnosis: high-grade serous adenocarcinoma (HGSC)'], 3: ['disease progression: early', 'disease progression: late']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and sample characteristics, this dataset appears to contain gene expression data
# from ovarian tumor biopsies of patients with high-grade serous adenocarcinoma (HGSC).
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Analyzing the sample characteristics dictionary:

# Trait (Ovarian Cancer): The key 3 contains "disease progression: early" and "disease progression: late"
# which can be used as a proxy for cancer outcome/progression
trait_row = 3

# Age: No explicit age information is provided in the sample characteristics
age_row = None

# Gender: Key 0 contains gender information, but it shows all patients are female
# Since there's only one value (female), it's constant and not useful for association studies
gender_row = None

# 2.2 Data Type Conversion
# Convert disease progression to binary (early: 1, late: 0)
def convert_trait(value):
    if value is None:
        return None
    # Extract the value part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'early' in value.lower():
        return 1  # Early disease progression
    elif 'late' in value.lower():
        return 0  # Late disease progression
    else:
        return None

# Define empty conversion functions for completeness, though they won't be used
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait="Disease_Progression",  # Using a more descriptive name for the trait
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM3596018': [1.0], 'GSM3596019': [0.0], 'GSM3596020': [1.0], 'GSM3596021': [1.0], 'GSM3596022': [0.0], 'GSM3596023': [0.0], 'GSM3596024': [1.0], 'GSM3596025': [0.0], 'GSM3596026': [0.0], 'GSM3596027': [1.0], 'GSM3596028': [1.0], 'GSM3596029': [1.0], 'GSM3596030': [0.0], 'GSM3596031': [0.0], 'GSM3596032': [1.0], 'GSM3596033': [1.0], 'GSM3596034': [1.0], 'GSM3596035': [1.0], 'GSM3596036': [1.0], 'GSM3596037': [1.0], 'GSM3596038': [1.0], 'GSM3596039': [1.0], 'GSM3596040': [0.0], 'GSM3596041': [0.0], 'GSM3596042': [0.0], 'GSM3596043': [0.0], 'GSM3596044': [0.0], 'GSM3596045': [0.0], 'GSM3596046': [0.0], 'GSM3596047': [0.0], 'GSM3596048': [0.0], 'GSM3596049': [0.0], 'GSM3596050': [0.0], 'GSM3596051': [0.0], 'GSM3596052': [0.0], 'GSM3596053': [0.0], 'GSM3596054': [1.0], 'GSM3596055': [0.0], 'GSM3596056': [0.0], 'GSM3596057': [0.0], 'GSM3596058': [0.0], 'GSM3596059': [1.0], 'GSM3596060': [1.0], 'GSM3596061': [0.0], 'GSM3596062': [0.0], 'GSM3596063': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be Affymetrix probeset IDs (with the '_st' suffix), not human gene symbols
# They need to be mapped to official gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# From the gene annotation preview, we see:
# - 'probeset_id' is the probe identifier (same format as gene expression data)
# - 'gene_assignment' contains gene symbols in a complex format

# Extract the mapping between probeset_id and gene symbols
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# 2. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# 3. Normalize gene symbols to standard format and handle duplicates
gene_data = normalize_gene_symbols_in_index(gene_data)

# 4. Preview the first few rows of converted gene expression data
print("Preview of gene expression data after mapping:")
print(gene_data.head())

# 5. Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Preview of gene expression data after mapping:
          GSM3596018  GSM3596019  GSM3596020  GSM3596021  GSM3596022  \
Gene                                                                   
A1BG         6.12260    6.339200    5.827900    6.005450    6.104400   
A1BG-AS1     1.82860    1.877100    1.690500    1.849950    1.934200   
A1CF         0.41832    0.420420    0.415560    0.425280    0.407020   
A2M          2.29640    2.325617    2.331117    2.338433    2.284167   
A2M-AS1      1.45670    1.468750    1.471750    1.457400    1.430400   

          GSM3596023  GSM3596024  GSM3596025  GSM3596026  GSM3596027  ...  \
Gene                                                                  ...   
A1BG         6.16070    6.341150    6.247650    6.018950     6.04270  ...   
A1BG-AS1     1.91520    2.008550    1.947150    1.910250     1.92280  ...   
A1CF         0.42452    0.441580    0.427800    0.420120     0.42674  ...   
A2M          2.30545    2.198733    2.278983    2.205517     2.

Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE126308.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data has already been normalized in the previous step
# No need to normalize again, we can directly use the gene_data from the previous step

# 2. Load the clinical data from the file since we need it for linking
# Loading clinical data from file to ensure it's available
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Disease_Progression")  # Using the trait name we assigned in step 2

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Disease_Progression")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Cohort contains ovarian cancer patients with early vs. late disease progression."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

For the feature 'Disease_Progression', the least common label is '1.0' with 20 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Disease_Progression' in this dataset is fine.



Linked data saved to ../../output/preprocess/Ovarian_Cancer/GSE126308.csv
